In [ ]:
# AI chat in this link: https://gemini.google.com/share/e62a45d84646

In [ ]:
import pandas as pd

# 1. Load the datasets
# Assuming the files are in the current working directory
oscar_df = pd.read_csv('the_oscar_nominees.csv')
imdb_df = pd.read_csv('imdb_top_1000.csv')

# 2. Filter Oscar nominees for movies from 1990 to the present
# We create a copy to avoid SettingWithCopy warnings
oscar_filtered = oscar_df[oscar_df['year_film'] >= 1990].copy()

# 3. Select the specific columns from the Oscar nominees list
oscar_subset = oscar_filtered[['year_film', 'release_date', 'year_ceremony', 'film', 'winner']]

# 4. Select the specific columns from the IMDB list
# We include 'Series_Title' to perform the merge
imdb_subset = imdb_df[['Series_Title', 'Genre', 'Runtime', 'Meta_score']]

# 5. Merge the lists
# We use a 'left' join to keep all Oscar nominees.
# Matching is done on the movie title ('film' in Oscar list, 'Series_Title' in IMDB list).
merged_df = pd.merge(oscar_subset, imdb_subset, left_on='film', right_on='Series_Title', how='left')

# 6. Clean up
# Remove the extra title column from IMDB
merged_df = merged_df.drop(columns=['Series_Title'])

# Display the first few rows to verify
print(merged_df.head())

# Save the result to a new CSV file
merged_df.to_csv('merged_oscar_imdb.csv', index=False)

   year_film release_date  year_ceremony                film  winner  \
0     1990.0   1990-02-23           1990     Cinema Paradiso    True   
1     1990.0   1990-05-25           1990   Jesus of Montreal   False   
2     1990.0   1990-11-09           1991  Dances With Wolves   False   
3     1990.0   1990-12-19           1991          Awakenings   False   
4     1990.0   1990-11-16           1991  Cyrano de Bergerac   False   

              Genre  Runtime  Meta_score  
0               NaN      NaN         NaN  
1               NaN      NaN         NaN  
2               NaN      NaN         NaN  
3  Biography, Drama  121 min        74.0  
4               NaN      NaN         NaN  


In [ ]:
# I insert the missing data manually and created 'best_picture_merged_complete.csv'

import pandas as pd

# Load the datasets
main_df = pd.read_csv('best_picture_merged_complete.csv')
oscar_df = pd.read_csv('the_oscar_nominees.csv')

# 1. Identify relevant categories in the Oscar nominees list
director_cats = ['DIRECTING', 'BEST DIRECTOR']
picture_cats = ['BEST PICTURE']

# 2. Extract nominees for Picture and Director to sets for efficient lookup
# We use a tuple of (film_name, ceremony_year) to ensure uniqueness
bp_nominees = oscar_df[oscar_df['category'].isin(picture_cats)][['film', 'year_ceremony']]
dir_nominees = oscar_df[oscar_df['category'].isin(director_cats)][['film', 'year_ceremony']]

bp_set = set(zip(bp_nominees['film'], bp_nominees['year_ceremony']))
dir_set = set(zip(dir_nominees['film'], dir_nominees['year_ceremony']))

# 3. Define a function to check if a movie is in both sets
def check_both(row):
    # Create the key from the row in the main dataframe
    key = (row['film'], row['year_ceremony'])

    is_bp = key in bp_set
    is_dir = key in dir_set

    return is_bp and is_dir

# 4. Apply the check to create the new column
main_df['Nominated_Both_Director_and_Picture'] = main_df.apply(check_both, axis=1)

# 5. Rearrange the list: Newer films first (year_film descending), then alphabetically (film ascending)
main_df = main_df.sort_values(by=['year_film', 'film'], ascending=[False, True])

# Save the result
main_df.to_csv('best_picture_director_merged.csv', index=False)

In [ ]:
import pandas as pd
import re

# 1. Load the datasets
gg_df = pd.read_csv('golden_globe_awards.csv')
# Start from the base merged list
main_df = pd.read_csv('best_picture_director_merged.csv')

# 2. Filter for WINNERS in relevant Picture categories
target_cats = [
    'Best Motion Picture - Drama',
    'Best Motion Picture - Musical or Comedy',
    'Picture - Musical',
    'Picture - Comedy',
    'Picture'
]

gg_winners = gg_df[
    (gg_df['category'].isin(target_cats)) &
    (gg_df['win'] == True)
].copy()

# 3. Clean and standardize Golden Globe titles
gg_winners['raw_title'] = gg_winners['film'].fillna(gg_winners['nominee'])

def clean_gg_title(t):
    t = str(t).strip()
    t = re.sub(r'\s*\(\d{4}\)$', '', t)  # Remove year

    # Handle "Title, The" format
    if t.endswith(', The'):
        t = 'The ' + t[:-5]
    elif t.endswith(', A'):
        t = 'A ' + t[:-3]
    elif t.endswith(', An'):
        t = 'An ' + t[:-4]

    t = re.sub(r'[^\w\s]', '', t) # Remove punctuation
    return t.strip().lower()

gg_winners['clean_title'] = gg_winners['raw_title'].apply(clean_gg_title)

# Create lookup set
gg_winner_lookup = set()
for _, row in gg_winners.iterrows():
    gg_winner_lookup.add((row['clean_title'], int(row['year_film'])))

# 4. Clean Main List titles
def clean_main_title(t):
    t = str(t).strip()
    if ' or ' in t:
        t = t.split(' or ')[0]
    t = re.sub(r'[^\w\s]', '', t)
    return t.strip().lower()

# 5. Apply check
def check_gg_win(row):
    t = clean_main_title(row['film'])
    y = int(row['year_film'])

    # Check exact and adjacent years
    for offset in [0, -1, 1]:
        if (t, y + offset) in gg_winner_lookup:
            return True
    return False

main_df['Golden_Globe_Picture_Winner'] = main_df.apply(check_gg_win, axis=1)

# 6. Sort and Save
main_df = main_df.sort_values(by=['year_film', 'film'], ascending=[False, True])
main_df.to_csv('best_picture_director_gg_winner_merged.csv', index=False)

In [1]:
import pandas as pd
import io

def clean_and_merge_data():
    # --- Step 1: Load and Clean 'Final_best_picture_data.csv' ---
    # The uploaded CSV file has a formatting issue where data rows are enclosed in double quotes.
    # We need to strip these outer quotes and unescape the inner quotes to parse it correctly.
    cleaned_lines = []

    with open('best_picture_director_gg_winner_merged.csv', 'r', encoding='utf-8') as f:
        lines = f.readlines()

        # Add the header line (usually it's correct)
        cleaned_lines.append(lines[0])

        # Process each data line
        for line in lines[1:]:
            line = line.strip()
            # Check if the line is wrapped in quotes
            if line.startswith('"') and line.endswith('"'):
                # Remove the first and last quote
                content = line[1:-1]
                # Replace double-double quotes ("") with a single quote (")
                # This fixes the escaped quotes inside fields like "Genre"
                content = content.replace('""', '"')
                cleaned_lines.append(content + '\n')
            else:
                # If the line is already correct, keep it
                cleaned_lines.append(line + '\n')

    # Create the DataFrame from the cleaned string content
    csv_content = "".join(cleaned_lines)
    df_final = pd.read_csv(io.StringIO(csv_content))

    # --- Step 2: Load 'PGA.csv' ---
    df_pga = pd.read_csv('PGA.csv')

    # --- Step 3: Prepare Data for Merging ---

    # Rename columns in PGA dataframe to match the Final dataframe
    # 'Tören Yılı' -> 'year_ceremony'
    # 'Film' -> 'film'
    df_pga = df_pga.rename(columns={'Tören Yılı': 'year_ceremony', 'Film': 'film'})

    # We only need to know which films are in the PGA list.
    # Create a subset with just the keys and a temporary flag.
    pga_winners = df_pga[['year_ceremony', 'film']].copy()
    pga_winners['is_pga_winner'] = True

    # Ensure consistent data types for merging keys
    df_final['year_ceremony'] = df_final['year_ceremony'].astype(int)
    pga_winners['year_ceremony'] = pga_winners['year_ceremony'].astype(int)

    # Strip any leading/trailing whitespace from film names to ensure better matching
    df_final['film'] = df_final['film'].astype(str).str.strip()
    pga_winners['film'] = pga_winners['film'].astype(str).str.strip()

    # --- Step 4: Merge Dataframes ---
    # Perform a Left Join: Keep all rows from df_final, match with pga_winners
    merged_df = pd.merge(df_final, pga_winners, on=['year_ceremony', 'film'], how='left')

    # --- Step 5: Create Boolean Column 'pga_winner' ---
    # Fill NaN values (films that didn't win PGA) with False, others True
    merged_df['pga_winner'] = merged_df['is_pga_winner'].fillna(False).astype(bool)

    # Drop the temporary column
    merged_df.drop(columns=['is_pga_winner'], inplace=True)

    # --- Step 6: Save and Inspect ---
    print("Merge Complete.")
    print(f"Total PGA Winners matched: {merged_df['pga_winner'].sum()}")
    print("\nFirst 5 rows with the new column:")
    print(merged_df[['year_ceremony', 'film', 'pga_winner']].head())

    # Save the final dataframe to a new CSV file
    merged_df.to_csv('Final_best_picture_data_with_PGA.csv', index=False)
    print("\nFile saved as 'Final_best_picture_data.csv'")

# Run the function
if __name__ == "__main__":
    clean_and_merge_data()

Merge Complete.
Total PGA Winners matched: 32

First 5 rows with the new column:
   year_ceremony                film  pga_winner
0           2025  A Complete Unknown       False
1           2025               Anora        True
2           2025            Conclave       False
3           2025      Dune: Part Two       False
4           2025        Emilia Pérez       False

File saved as 'Final_best_picture_data_with_PGA.csv'


/tmp/ipython-input-3498853093.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['pga_winner'] = merged_df['is_pga_winner'].fillna(False).astype(bool)


In [ ]:
# Since the golden globe data is until 2020, the data for the rest movies is added manually